# Implementing a Reactive Agent in AutoGen - SOLUTION

> **Learning Outcomes:**
> - Understand the purpose of a Reactive Agent
> - Create a Reactive Agent in AutoGen
> - Test the Reactive Agent in a simulated environment


## Introduction

In this lab, we will implement a reactive agent team that can respond to changes in its environment in real-time. This type of agent is useful in scenarios where the environment is dynamic and the agent needs to adapt its behavior based on the current state of the environment.

### AutoGen Framework
[AutoGen](https://microsoft.github.io/autogen/stable/index.html) is "a framework for building AI agents and applications" that is developed by Microsoft. It is designed to be a flexible and extensible framework that can be used to build a wide range of AI applications.

We will be working with the `autogen-agentchat` component which is designed for building conversational single and multi-agent systems. This component is built on top of the `autogen` framework and provides a simple way to build agents that can communicate with each other.


### The Scenario
A company wants to automate the process of collecting competitive intelligence from _NovaTech Innovations_. Previously, they have used the RSS feed of the competitor's website to get updates on new products and services. However, the RSS feed is full of irrelevant information and the company wants to filter out the noise and only get updates on new products and services.

Our agents will be responsible for monitoring the RSS feed of _NovaTech Innovations_ and filtering out the irrelevant information. The agent should be able to identify new products and services based on the information in the RSS feed and provide a summary of the updates to the company.


## Getting Started
Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.


In [1]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU \
    openai==1.95.* \
    autogen-agentchat==0.6.* \
    autogen-ext[openai]==0.6.* \
    feedparser==6.0.*

import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Note: you may need to restart the kernel to use updated packages.


Enter your OpenAI API key:  ········


## Core

### Step 1 - Read the RSS Feed

The first step in building the reactive agent is to read the RSS feed of _NovaTech Innovations_. We will use the `feedparser` library to read the RSS feed. The `feedparser` library is a Python library that can parse RSS and Atom feeds.

RSS feeds don't "push" updates to subscribers. Instead, subscribers need to "pull" updates from the feed at regular intervals. This could be setup as a cron job or a scheduled task. The agents can then be triggered based on the updates in the feed. While this is not a real-time solution, it is a common approach for monitoring RSS feeds. As you deploy agents in a production environment, you will likely find this type of setup common.

Feedparser is quite easy to use.

```python
feed = feedparser.parse('{the feed}}')
```

In the cell bellow, use the `feedparser` library to read the RSS feed of _NovaTech Innovations_ and print the title and summary of the feed entries. The RSS feed is stored in the `rss.xml` file. Explore the structure of the resulting feed to determine which fields may be relevant.

In [5]:
import feedparser

# YOUR CODE HERE
# [SOLUTION]
feed = feedparser.parse("rss.xml")
feed['entries'][0]

{'title': 'NovaTech Launches Revolutionary AI Chip',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': '',
  'value': 'NovaTech Launches Revolutionary AI Chip'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://novatechinnovations.example.com/news/ai-chip-launch'}],
 'link': 'https://novatechinnovations.example.com/news/ai-chip-launch',
 'summary': 'NovaTech announces the release of its groundbreaking AI processing chip, the "NeuralSpark."',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': '',
  'value': 'NovaTech announces the release of its groundbreaking AI processing chip, the "NeuralSpark."'},
 'published': 'Tue, 24 Oct 2023 09:30:00 GMT',
 'published_parsed': time.struct_time(tm_year=2023, tm_mon=10, tm_mday=24, tm_hour=9, tm_min=30, tm_sec=0, tm_wday=1, tm_yday=297, tm_isdst=0)}

### Step 2 - Implement the Agents

Now that we have read the RSS feed, we can implement the reactive agents. The agents should be able to filter out irrelevant information and identify important competitive intelligence in the feed.

Once the agents have identified the important information, they should be able to provide a short summary of the information and why it is important.

In the cell below, implement the reactive agents. The boilerplate code to setup a round-robin chat between the agents is provided. You can add your own logic to the agents to filter out irrelevant information and identify important competitive intelligence.

In [6]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

model_client = OpenAIChatCompletionClient(model="gpt-4.1-mini")
termination = TextMentionTermination("FILTER") | TextMentionTermination("TERMINATE")

# YOUR CODE HERE
# [SOLUTION]
importance_agent = AssistantAgent(
    name="importance_agent",
    system_message="""
You are a competitive intelligence expert.
Your job is to assess the importance of a news updated.
Rate the importance as: High, Medium, Low
If an article is not important, reply FILTER to end the conversation.
""".strip(),
    model_client=model_client,
)

summarization_agent = AssistantAgent(
    name="summarization_agent",
    system_message="""
You are a competitive intelligence expert.
Your job is to summarize new from competitors.
Keep the summary brief and informative.
Avoid hype (e.g., revolutionary, cutting-edge) and copying the article.
Use objective language.
Use the importance rating from the previous agent.

Use this format:

**[Importance] Summarizing Title (Date)**
Summary: [1 sentence summary]
Recommended Actions:
- [1-3 action items, each 15 words or less]
""".strip(),
    model_client=model_client,
)

review_agent = AssistantAgent(
    name="review_agent",
    system_message="""
You are a business analyst.
Your job is to review the importance and summary of a news update.
If you disagree with the summary, ask for a re-write.
For each disagreement, provide a reason.

Reply TERMINATE to accept the importance and summary as written.
""".strip(),
    model_client=model_client,
)
# [/SOLUTION]

team = RoundRobinGroupChat([
    # PLACE YOUR AGENTS HERE
    importance_agent, summarization_agent, review_agent
], termination_condition=termination, max_turns=12)


stream = team.run_stream(task=str(feed['entries'][0]))
await Console(stream)

---------- TextMessage (user) ----------
{'title': 'NovaTech Launches Revolutionary AI Chip', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': 'NovaTech Launches Revolutionary AI Chip'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://novatechinnovations.example.com/news/ai-chip-launch'}], 'link': 'https://novatechinnovations.example.com/news/ai-chip-launch', 'summary': 'NovaTech announces the release of its groundbreaking AI processing chip, the "NeuralSpark."', 'summary_detail': {'type': 'text/html', 'language': None, 'base': '', 'value': 'NovaTech announces the release of its groundbreaking AI processing chip, the "NeuralSpark."'}, 'published': 'Tue, 24 Oct 2023 09:30:00 GMT', 'published_parsed': time.struct_time(tm_year=2023, tm_mon=10, tm_mday=24, tm_hour=9, tm_min=30, tm_sec=0, tm_wday=1, tm_yday=297, tm_isdst=0)}
---------- TextMessage (importance_agent) ----------
High
---------- TextMessage (summarization_agent) ----------
**[H

TaskResult(messages=[TextMessage(id='8f1ca5a7-bb2b-4672-9d93-af647c684a4b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 14, 29, 42, 557771, tzinfo=datetime.timezone.utc), content='{\'title\': \'NovaTech Launches Revolutionary AI Chip\', \'title_detail\': {\'type\': \'text/plain\', \'language\': None, \'base\': \'\', \'value\': \'NovaTech Launches Revolutionary AI Chip\'}, \'links\': [{\'rel\': \'alternate\', \'type\': \'text/html\', \'href\': \'https://novatechinnovations.example.com/news/ai-chip-launch\'}], \'link\': \'https://novatechinnovations.example.com/news/ai-chip-launch\', \'summary\': \'NovaTech announces the release of its groundbreaking AI processing chip, the "NeuralSpark."\', \'summary_detail\': {\'type\': \'text/html\', \'language\': None, \'base\': \'\', \'value\': \'NovaTech announces the release of its groundbreaking AI processing chip, the "NeuralSpark."\'}, \'published\': \'Tue, 24 Oct 2023 09:30:00 GMT\', \'published_pa

### Step 3 - Test the Agents

Finally, lets test the agents in a simulated environment. You can simulate the environment by creating a list of messages that represent the updates in the RSS feed. The agents should be able to process the messages and provide a summary of the important information.

In the cell below, create a function that formats the RSS feed entries as messages. Then test the agents by passing the messages to the agents and printing the summary of the important information.

In [7]:
from IPython.display import display, Markdown

# YOUR CODE HERE
# [SOLUTION]
def format_update(update):
    return f"**{update['title']} ({update['published']})**\n{update['description']}\n"

for update in feed['entries'][:5]:
    # print(format_update(update))
    result = await team.run(task=format_update(update))

    if 'FILTER' in result.stop_reason:
        print('Skipping message')
        continue

    display(Markdown(result.messages[-2].content))

**[High] NovaTech Launches AI Processing Chip "NeuralSpark" (Oct 24, 2023)**
Summary: NovaTech introduced the "NeuralSpark," a new AI processing chip aimed at improving AI processing efficiency.
Recommended Actions:
- Assess NeuralSpark's features versus existing AI chips in our product line.
- Track competitive response and potential market impact.
- Explore collaboration or counter-development strategies.

**[Medium] NovaTech Partners with Quantum Computing Firm QuantumLeap (Oct 23, 2023)**
Summary: NovaTech formed a strategic partnership with QuantumLeap Technologies to investigate quantum computing applications.
Recommended Actions:
- Monitor partnership developments and potential technology breakthroughs.
- Evaluate implications for our quantum and AI strategies.
- Consider initiating dialogues with QuantumLeap for early insights.

**[Medium] NovaTech Stock Price Rises After Positive Earnings (Oct 23, 2023)**
Summary: NovaTech's shares increased notably in response to a strong quarterly earnings report.
Recommended Actions:
- Analyze earnings details for growth drivers and financial health.
- Assess impact on competitor market positioning.
- Review potential effects on investor sentiment and funding availability.

Skipping message
Skipping message


Congratulations! You have successfully implemented a reactive agent team that can respond to changes in the environment (the rss feed). You can now deploy the agents in a production environment to automate the process of collecting competitive intelligence from _NovaTech Innovations_.

## Bonus Challenge 1 - Update Aggregator

The agents are currently only able to process one message at a time. This may not be the best way to provide information to the company. Create an aggregator agent that can aggregate the competitive intelligence from the other agents and provide a summary of the important information.

This agent could be triggered to run at regular intervals and provide a summary of the competitive intelligence that has been collected by the other agents.

In [ ]:
# YOUR CODE HERE

## Bonus Challenge 2 - File System Monitor

As part of a legacy workflow, the company has a folder where they store documents related to _NovaTech Innovations_. Create a file system monitor agent that can monitor the folder for new documents and provide a summary of the important information in the documents.

<div class="alert alert-block alert-info">
<b>Tip:</b> In production, you would likely use a library like `watchdog` to monitor the file system for changes. In this lab, you can just check the folder for new files at regular intervals.
</div>

In [ ]:
# YOUR CODE HERE